# Naver Movie Dataset

17011724 최시우 / 17011739 이재훈 / 17011745 최영은

In [1]:
from bs4 import BeautifulSoup
import numpy as np
import requests 
from urllib.request import urlopen
import math
import pymysql

### 네이버 영화에서 필요한 데이터 크롤링

In [2]:
genre_ = ["드라마", "판타지", "서부", "공포", "멜로/로맨스", "모험",
          "스릴러", "느와르", "컬트", "다큐멘터리", "코미디", "가족",
          "미스터리", "전쟁", "애니메이션", "범죄", "뮤지컬", "SF", "액션",
          "무협", "에로", "서스펜스", "서사", "블랙코미디", "실험", "공연실황"]

In [3]:
def movie_data(start, number):
    code_list = []
    movie_list = []
    genre_list =[]
    score_list = []
    grp_list = []
    html=[]
    cnt=0 #평점과 감상포인트가 모두 등록된 영화 개수 카운트
    k=0.1 # 계산을 위한 값
    for n in range (start, (start+number)):
        url = 'https://movie.naver.com/movie/bi/mi/point.nhn?code='+str(n)#영화 코드가 변함에 따라 웹페이지가 변함
        resp = requests.get(url)
        html.append(BeautifulSoup(resp.content, 'html.parser'))
          
    for i in range(number):
        grp_point = html[i].find('ul', {'class': 'grp_point'})
        netizen_score = html[i].find('div', {'id': 'netizen_point_tab_inner'})
        if (grp_point != None) & (netizen_score != None):
            tmp = grp_point.findAll('span')
            for x in range (5):
                tmp[x]=tmp[x].string  #숫자부분만 출력
                
            if '0%' not in tmp:
                code_list.append(i+start)#코드번호 추출
                movie_list.append(html[i].find('h3', {'class': 'h_movie'}).find('a').string) #영화이름 추출
                genre_list.append(html[i].find('dl', {'class': 'info_spec'}).findAll(string=genre_))#영화장르 추출
                grp_list.append(tmp)#영화 관람 포인트 추출
                scr_list = netizen_score.findAll('em')  #영화 네티즌 평점 추출    
                
                scr_list[0]=float(scr_list[0].string) #영화 네티즌 평점을 float형으로 고치기 위한 코드
                score = scr_list[0]
                for a in range (2):
                    scr_list[a+1]=scr_list[a+2]
                    scr_list[a+1]=float(scr_list[a+1].string)
                    score = score + scr_list[a+1] * k
                    k=k/10
                score_list.append(score)
                cnt=cnt+1
        k=0.1   
    movie_data_set = ([[0]*9 for i in range(cnt)])
    for i in range (cnt):
        movie_data_set[i][0] = code_list[i] #영화코드
        movie_data_set[i][1] = movie_list[i] #영화이름
        movie_data_set[i][2] = genre_list[i] #영화장르
        
        movie_data_set[i][3] = grp_list[i][0] #영화 관람포인트
        movie_data_set[i][4] = grp_list[i][1]
        movie_data_set[i][5] = grp_list[i][2]
        movie_data_set[i][6] = grp_list[i][3]
        movie_data_set[i][7] = grp_list[i][4]
        
        movie_data_set[i][8] = score_list[i] #영화 평점
        
    for i in range (cnt): #영화 관람포인트를 float형으로 고치기 위한 코드
        for j in range(3,8):
            if '%' not in movie_data_set[i][j][0:2]:
                movie_data_set[i][j]=float(movie_data_set[i][j][0:2])/100
            else:
                movie_data_set[i][j]=float(movie_data_set[i][j][0:1])/100
   
    return movie_data_set

In [4]:
start=int(input()) #영화 조사 시작
number=int(input()) #조사할 영화 수
DATASET=movie_data(start,number)
start=start+number #조사가 끝난후 시작점 갱신
DATASET

167600
100


[[167602,
  '나를 차버린 스파이',
  ['액션', '코미디'],
  0.19,
  0.38,
  0.26,
  0.1,
  0.07,
  7.930000000000001],
 [167605, '터미네이터: 다크 페이트', ['SF', '액션'], 0.29, 0.21, 0.19, 0.22, 0.09, 8.62],
 [167613, '조커', ['스릴러', '드라마'], 0.2, 0.34, 0.17, 0.14, 0.14, 8.65],
 [167615, '이스케이프 플랜3', ['액션'], 0.17, 0.22, 0.22, 0.17, 0.22, 2.67],
 [167616, '그때 그들', ['드라마', '코미디'], 0.29, 0.29, 0.14, 0.14, 0.14, 7.28],
 [167629, '어쩌다, 결혼', ['멜로/로맨스', '코미디'], 0.1, 0.3, 0.3, 0.15, 0.15, 4.46],
 [167633, '마담 싸이코', ['스릴러'], 0.29, 0.35, 0.18, 0.1, 0.08, 6.81],
 [167635, '말레피센트 2', ['판타지'], 0.19, 0.23, 0.18, 0.31, 0.09, 7.619999999999999],
 [167638, '완벽한 타인', ['드라마', '코미디'], 0.24, 0.34, 0.32, 0.05, 0.04, 8.62],
 [167647, '풀잎들', ['드라마'], 0.28, 0.28, 0.28, 0.06, 0.11, 7.609999999999999],
 [167651, '극한직업', ['코미디'], 0.28, 0.38, 0.2, 0.06, 0.07, 8.52],
 [167653,
  '광대들: 풍문조작단',
  ['드라마'],
  0.2,
  0.37,
  0.27,
  0.11,
  0.05,
  6.359999999999999],
 [167657, '명탐정 피카츄', ['액션', '모험', '코미디'], 0.21, 0.1, 0.21, 0.41, 0.07, 8.09],
 [1

In [5]:
number=int(input()) #조사할 영화수(추가조사)
DATASET.extend(movie_data(start,number)) #갱신된 시작점에서 조사할 영화수만큼 조사->불필요한 수집 방지
start=start+number
DATASET

100


[[167602,
  '나를 차버린 스파이',
  ['액션', '코미디'],
  0.19,
  0.38,
  0.26,
  0.1,
  0.07,
  7.930000000000001],
 [167605, '터미네이터: 다크 페이트', ['SF', '액션'], 0.29, 0.21, 0.19, 0.22, 0.09, 8.62],
 [167613, '조커', ['스릴러', '드라마'], 0.2, 0.34, 0.17, 0.14, 0.14, 8.65],
 [167615, '이스케이프 플랜3', ['액션'], 0.17, 0.22, 0.22, 0.17, 0.22, 2.67],
 [167616, '그때 그들', ['드라마', '코미디'], 0.29, 0.29, 0.14, 0.14, 0.14, 7.28],
 [167629, '어쩌다, 결혼', ['멜로/로맨스', '코미디'], 0.1, 0.3, 0.3, 0.15, 0.15, 4.46],
 [167633, '마담 싸이코', ['스릴러'], 0.29, 0.35, 0.18, 0.1, 0.08, 6.81],
 [167635, '말레피센트 2', ['판타지'], 0.19, 0.23, 0.18, 0.31, 0.09, 7.619999999999999],
 [167638, '완벽한 타인', ['드라마', '코미디'], 0.24, 0.34, 0.32, 0.05, 0.04, 8.62],
 [167647, '풀잎들', ['드라마'], 0.28, 0.28, 0.28, 0.06, 0.11, 7.609999999999999],
 [167651, '극한직업', ['코미디'], 0.28, 0.38, 0.2, 0.06, 0.07, 8.52],
 [167653,
  '광대들: 풍문조작단',
  ['드라마'],
  0.2,
  0.37,
  0.27,
  0.11,
  0.05,
  6.359999999999999],
 [167657, '명탐정 피카츄', ['액션', '모험', '코미디'], 0.21, 0.1, 0.21, 0.41, 0.07, 8.09],
 [1

In [6]:
#네이버 영화에서 추출해온 DATASET의 장르를 따로 분리(데이터베이스에서 용이하게 다루기 위함)
genre_list = []
n=len(DATASET)
for i in range (n):
    tmp=DATASET[i].pop(2)
    genre_list.append(tmp)
    while( np.size(genre_list[i])<3):
        genre_list[i].append(None)
    
genre_list

[['액션', '코미디', None],
 ['SF', '액션', None],
 ['스릴러', '드라마', None],
 ['액션', None, None],
 ['드라마', '코미디', None],
 ['멜로/로맨스', '코미디', None],
 ['스릴러', None, None],
 ['판타지', None, None],
 ['드라마', '코미디', None],
 ['드라마', None, None],
 ['코미디', None, None],
 ['드라마', None, None],
 ['액션', '모험', '코미디'],
 ['판타지', '드라마', None],
 ['드라마', None, None],
 ['드라마', None, None]]

In [7]:
DATASET

[[167602, '나를 차버린 스파이', 0.19, 0.38, 0.26, 0.1, 0.07, 7.930000000000001],
 [167605, '터미네이터: 다크 페이트', 0.29, 0.21, 0.19, 0.22, 0.09, 8.62],
 [167613, '조커', 0.2, 0.34, 0.17, 0.14, 0.14, 8.65],
 [167615, '이스케이프 플랜3', 0.17, 0.22, 0.22, 0.17, 0.22, 2.67],
 [167616, '그때 그들', 0.29, 0.29, 0.14, 0.14, 0.14, 7.28],
 [167629, '어쩌다, 결혼', 0.1, 0.3, 0.3, 0.15, 0.15, 4.46],
 [167633, '마담 싸이코', 0.29, 0.35, 0.18, 0.1, 0.08, 6.81],
 [167635, '말레피센트 2', 0.19, 0.23, 0.18, 0.31, 0.09, 7.619999999999999],
 [167638, '완벽한 타인', 0.24, 0.34, 0.32, 0.05, 0.04, 8.62],
 [167647, '풀잎들', 0.28, 0.28, 0.28, 0.06, 0.11, 7.609999999999999],
 [167651, '극한직업', 0.28, 0.38, 0.2, 0.06, 0.07, 8.52],
 [167653, '광대들: 풍문조작단', 0.2, 0.37, 0.27, 0.11, 0.05, 6.359999999999999],
 [167657, '명탐정 피카츄', 0.21, 0.1, 0.21, 0.41, 0.07, 8.09],
 [167697, '신과함께-인과 연', 0.18, 0.28, 0.3, 0.17, 0.07, 7.71],
 [167699, '말모이', 0.19, 0.3, 0.38, 0.08, 0.06, 9.09],
 [167787, '허스토리', 0.2, 0.2, 0.33, 0.13, 0.13, 9.29]]

### 크롤링한 데이터를 바탕으로 DB연동

In [9]:
# Movie 테이블에 영화 데이터 삽입

conn = pymysql.connect(host='localhost', user='root', password='0000', db='moviers', charset='utf8')
curs = conn.cursor()

sql = "insert into movie(영화번호, 영화명, 연출, 연기, 스토리, 영상미, OST, 평점) values (%s, %s, %s, %s, %s, %s, %s, %s)"
curs.executemany(sql, DATASET)

conn.commit()

In [10]:
# MovieGenre 테이블에 번호, 영화번호, 장르 데이터 삽입

conn = pymysql.connect(host='localhost', user='root', password='0000',db='moviers', charset='utf8')
curs = conn.cursor()

for i in range(0,n):
    sql = "insert into moviegenre(번호, 영화번호, 장르1, 장르2, 장르3) values (%s, %s, %s, %s, %s)"
    curs.execute(sql, (i+1, DATASET[i][0], genre_list[i][0], genre_list[i][1], genre_list[i][2]))

conn.commit()

In [13]:
# WatchingPoint 테이블에서 유저의 관람포인트 세 개를 추출

conn = pymysql.connect(host='localhost', user='root', password='0000', db='moviers', charset='utf8')
curs = conn.cursor()

sql = "select 관람포인트1, 관람포인트2, 관람포인트3 from watchingpoint"
curs.execute(sql)

rows = curs.fetchall()

user_input = []
for i in range (0,3):
    user_input.append(rows[0][i])
    user_input[i] = user_input[i] + 2

### 사용자가 선택한 관람포인트와 가중평점을 곱하여 영화를 추천

In [14]:
#사용자가 임의로 입력한 관람포인트 순서대로 관람포인트 점수와 가중치를 값을 더하여 
#영화 평점과 곱한 새로운 가중평점순으로 영화를 추천해줌
Sum=0
user_choice=[]
#user_input=[] #사용자가 선호하는 관람포인트를 인덱스로 입력 받음
c_rating=[] #가중평점
weight=[2.5, 1.5, 1] # 가중치. 
Index=[]

#for i in range (3):
#    user_input.append(int(input()))
#    user_input[i]=user_input[i]+2
for i in range (n):
    for j in range(3):
        user_choice.append(DATASET[i][user_input[j]]*weight[j])
        Sum = Sum + user_choice[j]
    c_rating.append(round(Sum*DATASET[i][7],3))
    Sum=0 #연산값 초기화
    user_choice=[] #배열 초기화
for i in range (n):                 
    tmp=c_rating.index(max(c_rating))
    Index.append(tmp)
    c_rating[tmp]=-1
    
r_DATASET =([[] for i in range(n)])
for i in range (n):
    r_DATASET[i]=DATASET[Index[i]]
#movie_rating=np.array(movie_rating)
r_DATASET

[[167651, '극한직업', 0.28, 0.38, 0.2, 0.06, 0.07, 8.52],
 [167638, '완벽한 타인', 0.24, 0.34, 0.32, 0.05, 0.04, 8.62],
 [167699, '말모이', 0.19, 0.3, 0.38, 0.08, 0.06, 9.09],
 [167647, '풀잎들', 0.28, 0.28, 0.28, 0.06, 0.11, 7.609999999999999],
 [167605, '터미네이터: 다크 페이트', 0.29, 0.21, 0.19, 0.22, 0.09, 8.62],
 [167787, '허스토리', 0.2, 0.2, 0.33, 0.13, 0.13, 9.29],
 [167602, '나를 차버린 스파이', 0.19, 0.38, 0.26, 0.1, 0.07, 7.930000000000001],
 [167613, '조커', 0.2, 0.34, 0.17, 0.14, 0.14, 8.65],
 [167633, '마담 싸이코', 0.29, 0.35, 0.18, 0.1, 0.08, 6.81],
 [167616, '그때 그들', 0.29, 0.29, 0.14, 0.14, 0.14, 7.28],
 [167697, '신과함께-인과 연', 0.18, 0.28, 0.3, 0.17, 0.07, 7.71],
 [167653, '광대들: 풍문조작단', 0.2, 0.37, 0.27, 0.11, 0.05, 6.359999999999999],
 [167635, '말레피센트 2', 0.19, 0.23, 0.18, 0.31, 0.09, 7.619999999999999],
 [167657, '명탐정 피카츄', 0.21, 0.1, 0.21, 0.41, 0.07, 8.09],
 [167629, '어쩌다, 결혼', 0.1, 0.3, 0.3, 0.15, 0.15, 4.46],
 [167615, '이스케이프 플랜3', 0.17, 0.22, 0.22, 0.17, 0.22, 2.67]]

영화의 유사도를 코사인 메소드를 통해 구함

In [15]:
movie_rating =([[] for i in range(n)])
for i in range (n):
    for j in range(2,7):
        movie_rating[i].append(r_DATASET[i][j])

movie_rating=np.array(movie_rating)

#print(type(movie_rating[0][0]))
#print(movie_rating[0][0])
movie_rating
def cos(a):
    #a=a*10 #편의로 10을 곱함
    #a=a[:,1:6] # 행렬(movie_rating의 평점과 가중평점을 제외)
    sim=np.full((n,n),0.0)# 초기화
    for i in range(0,n):
        for j in range(i,n):
            InnerDot=np.dot(a[i,],a[j,])
            Normi=np.linalg.norm(a[i,])
            Normj=np.linalg.norm(a[j,])
            sim[i,j]=InnerDot/(Normi*Normj)
            sim[i,j]=round(sim[i,j],3)
            sim[j,i]=sim[i,j]
            
    return sim

In [16]:
sim=cos(movie_rating)#1에 가까울수록 영화의 관람포인트가 유사함.
sim

array([[1.   , 0.966, 0.914, 0.966, 0.893, 0.874, 0.975, 0.964, 0.995,
        0.957, 0.919, 0.973, 0.814, 0.62 , 0.877, 0.856],
       [0.966, 1.   , 0.984, 0.979, 0.871, 0.935, 0.98 , 0.922, 0.952,
        0.895, 0.959, 0.984, 0.798, 0.635, 0.919, 0.853],
       [0.914, 0.984, 1.   , 0.961, 0.857, 0.967, 0.961, 0.895, 0.902,
        0.852, 0.975, 0.967, 0.81 , 0.677, 0.95 , 0.875],
       [0.966, 0.979, 0.961, 1.   , 0.919, 0.958, 0.958, 0.94 , 0.965,
        0.944, 0.951, 0.96 , 0.826, 0.682, 0.912, 0.909],
       [0.893, 0.871, 0.857, 0.919, 1.   , 0.915, 0.882, 0.923, 0.931,
        0.96 , 0.93 , 0.893, 0.958, 0.892, 0.858, 0.919],
       [0.874, 0.935, 0.967, 0.958, 0.915, 1.   , 0.917, 0.898, 0.884,
        0.883, 0.972, 0.923, 0.872, 0.788, 0.953, 0.947],
       [0.975, 0.98 , 0.961, 0.958, 0.882, 0.917, 1.   , 0.97 , 0.967,
        0.923, 0.968, 0.998, 0.852, 0.668, 0.951, 0.889],
       [0.964, 0.922, 0.895, 0.94 , 0.923, 0.898, 0.97 , 1.   , 0.974,
        0.975, 0.942, 0.96

In [17]:
def return_index(i):
    b=[]
    a=list(sim[i])
    a[i]=-1 #a[i]는 비교하려는 영화와 같음으로 -1로 초기화

    for i in range (10):   #인덱스가 1인 영화와 유사한 영화를 평점과 상관없이                     
        tmp=a.index(max(a))#10개를 선정하여 리턴
        b.append(tmp)
        a[tmp]=-1
    return b

In [18]:
i=int(input()) #사용자가 선택한 영화의 인덱스
k=return_index(i)#유사도가 높은 영화의 인덱스를 저장함
r_movie = [] #영화 추천리스트 생성
tmp=[]
for i in range(10): 
    r_movie.append(DATASET[k[i]])   
r_movie

1


[[167613, '조커', 0.2, 0.34, 0.17, 0.14, 0.14, 8.65],
 [167653, '광대들: 풍문조작단', 0.2, 0.37, 0.27, 0.11, 0.05, 6.359999999999999],
 [167633, '마담 싸이코', 0.29, 0.35, 0.18, 0.1, 0.08, 6.81],
 [167615, '이스케이프 플랜3', 0.17, 0.22, 0.22, 0.17, 0.22, 2.67],
 [167602, '나를 차버린 스파이', 0.19, 0.38, 0.26, 0.1, 0.07, 7.930000000000001],
 [167651, '극한직업', 0.28, 0.38, 0.2, 0.06, 0.07, 8.52],
 [167638, '완벽한 타인', 0.24, 0.34, 0.32, 0.05, 0.04, 8.62],
 [167629, '어쩌다, 결혼', 0.1, 0.3, 0.3, 0.15, 0.15, 4.46],
 [167635, '말레피센트 2', 0.19, 0.23, 0.18, 0.31, 0.09, 7.619999999999999],
 [167699, '말모이', 0.19, 0.3, 0.38, 0.08, 0.06, 9.09]]

In [19]:
# 구현한 추천시스템을 통해 나온 결과를 recommendation 테이블에 데이터 삽입

conn = pymysql.connect(host='localhost', user='root', password='0000', db='moviers', charset='utf8')
curs = conn.cursor()

sql = "insert into recommendation(영화번호, 영화명, 연출, 연기, 스토리, 영상미, OST, 평점) values (%s, %s, %s, %s, %s, %s, %s, %s)"
curs.executemany(sql, r_movie)

conn.commit()